### Test case LVV-T153: Verify implementation of Engineering and Facility Database Archive

Demonstrate Engineering and Facilities Data (images, associated metadata, and observatory environment and control data) are archived.

#### Discussion:

This can be verified using the copy of the Engineering Facilities Database (EFD) that is regularly transferred from the summit to the U.S. Data Facility (USDF). This USDF version of the EFD is the "archived" version.

To demonstrate that EFD data are being archived, we will examine EFD topics that are typically populated during on-sky observing, selecting an observing night that was more than one month ago (specifically, 5 May 2025).

#### Look up the current time and print it to the screen:

In [1]:
import datetime

# Note that Rubin Observatory reports quantities in TAI, but for
#   our present purposes, the 37-second difference between UTC and TAI
#   is unimportant.
time_now = datetime.datetime.now(datetime.UTC)
print(time_now)

2025-06-13 22:58:17.178100+00:00


The current UTC time is June 13, 2025, so our selected observing date of May 5, 2025 is more than a month prior to this test. (Note: the day_obs is the local date in Chile on which the observing night began.)

In [2]:
from astropy.time import Time
from astropy.table import Table
import numpy as np

from lsst_efd_client import EfdClient 
from IPython.display import Markdown, display, display_markdown

Instantiate the EFD client, pointed to the USDF EFD, and confirm that it is accessible.

In [3]:
client = EfdClient('usdf_efd')
print(client)
assert client is not None

display_markdown("Engineering Facilities Database is accessible", raw=True)

Engineering Facilities Database is accessible

#### Examine the list of available EFD topics

Print the length of the list and a randomly-selected subset of topics.

In [9]:
topics = await client.get_topics()

print('Number of EFD topics: ', len(topics))

nsamp = 50
random_topics = np.random.choice(topics, nsamp)

print(f"{nsamp} randomly selected topics:")
print(random_topics)

Number of EFD topics:  3090
50 randomly selected topics:
['lsst.sal.MTMount.logevent_deployablePlatformsMotionState'
 'lsst.sal.MTAOS.command_resetOffsetDOF'
 'lsst.sal.WeatherForecast.logevent_summaryState'
 'lsst.sal.MTCamera.logevent_quadbox_PDU_5V_LimitsConfiguration'
 'lsst.sal.CCArchiver.command_start'
 'lsst.sal.MTCamera.logevent_utiltrunk_PeriodicTasks_UtMaq20_check_status_timersConfiguration'
 'lsst.sal.Scheduler.logevent_summaryState'
 'lsst.sal.ATCamera.logevent_bonn_shutter_PeriodicTasks_GeneralConfiguration'
 'lsst.sal.FiberSpectrograph.logevent_errorCode'
 'lsst.sal.OCPS.logevent_configurationsAvailable'
 'lsst.sal.MTPtg.command_poriginOffset'
 'lsst.sal.MTOODS.logevent_summaryState'
 'lsst.sal.MTPtg.logevent_softwareVersions'
 'lsst.sal.MTM2.logevent_configurationFiles'
 'lsst.sal.MTCamera.focal_plane_Reb'
 'lsst.sal.Scheduler.logevent_largeFileObjectAvailable'
 'lsst.sal.MTM1M3.logevent_powerStatus' 'lsst.sal.ATPtg.logevent_iers'
 'lsst.sal.ATCamera.command_stop' 'lsst.

This list contains topics pertaining to AuxTel (AT*), the Simonyi Telescope ("MainTel" -- denoted MT*), including the telescope mount (MTMount), the dome (MTDome), the AOS system (MTAOS), and the HVAC system, and LSSTCam (MTCamera), among others. We also note that topics include commands, logevents, and telemetry from sensors (e.g., "chillerFanSpeeds", "MTM2.axialEncoderPositions").

#### Extract some EFD data and confirm that it is reasonable

A variety of topics are available. Now we arbitrarily select a few topics and demonstrate that their data is intact and well-formed.

First, define a time range to query (in this case, 03:00-03:30UTC on 2025-05-05), using the Astropy `Time` package.

In [5]:
# Get data within a particular time range
start = Time("2025-05-05T03:00:00Z", scale="utc")
end = Time("2025-05-05T03:30:00Z", scale="utc")

Create a dictionary of EFD topics and fields to retrieve for each of them. This is an arbitrary list, but includes topics related to the telescope mount, environmental sensors, AOS system, M1M3 mirror accelerometers, telescope rotator, and LSSTCam. The retrieved columns include telemetry, commands, and log events.

In [6]:
topic_fields_dict = {'lsst.sal.MTMount.azimuth':['actualPosition', 'actualPositionTimestamp'],
                     'lsst.sal.MTMount.elevation':['actualPosition', 'actualPositionTimestamp'],
                     'lsst.sal.ESS.temperature':['sensorName', 'temperatureItem0'],
                     'lsst.sal.MTAOS.logevent_cameraHexapodCorrection':['u', 'v', 'w', 'x', 'y', 'z'],
                     'lsst.sal.MTAOS.logevent_wavefrontError':['nollZernikeValues7', 'nollZernikeValues8', 'nollZernikeValues9', 'sensorId'], 
                     'lsst.sal.MTM1M3.accelerometerData':['accelerometer0', 'accelerometer1', 'timestamp'],
                     'lsst.sal.MTRotator.logevent_target':['position', 'tai', 'velocity'],
                     'lsst.sal.MTRotator.logevent_lowFrequencyVibration':['frequency'],
                     'lsst.sal.MTCamera.command_takeImages':['expTime', 'numImages', 'shutter'],
                     'lsst.sal.MTCamera.logevent_startIntegration':['exposureTime','imageDate', 'imageIndex', 'imageName', 'imageNumber']
                    }

Query each of the topics from the dict above in turn, and print the data in the first 3 rows of their requested fields to the screen. Note that we initially retrieve all columns of the results (specified by the ['*'] wildcard), and then sub-select a specific list of columns from the list.

In [7]:
for topicfield in topic_fields_dict.keys():
    print(f"*** Retrieving {topicfield} ***\n")

    results = await client.select_time_series(topicfield, ['*'], start, end)
    # Sub-select only the desired columns:
    results = results[topic_fields_dict[topicfield]]

    print(Table.from_pandas(results, index=True)[:3],'\n')

*** Retrieving lsst.sal.MTMount.azimuth ***

             index                actualPosition  actualPositionTimestamp
-------------------------------- ---------------- -----------------------
2025-05-05 03:00:00.017759+00:00 16.6336992840165        1746414036.88586
2025-05-05 03:00:00.118609+00:00 16.6328785412205        1746414036.98565
2025-05-05 03:00:00.183883+00:00 16.6324684927823        1746414037.03552 

*** Retrieving lsst.sal.MTMount.elevation ***

             index                actualPosition  actualPositionTimestamp
-------------------------------- ---------------- -----------------------
2025-05-05 03:00:00.018120+00:00 60.4935636867039        1746414036.91525
2025-05-05 03:00:00.119741+00:00 60.4936641347571        1746414037.01558
2025-05-05 03:00:00.184244+00:00  60.493717586413        1746414037.06586 

*** Retrieving lsst.sal.ESS.temperature ***

             index                     sensorName        temperatureItem0
-------------------------------- ------------

## Results

We see that the EFD at USDF contains a wide variety of information taken directly from the SAL telemetry stream for an observing night over a month ago. This includes telemetry from all subsystems of the telescope and the dome, environmental information, and details about the camera.

We have demonstrated that the EFD is archived at the USDF.

The result of this test is **PASS**.